# Cut-in detection

In this notebook, the cut ins are detected. 

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp, kendalltau, spearmanr
from domain_model import StateVariable
from databaseemulator import DataBaseEmulator
from data_handler import DataHandler
from stats import KDE

In [ ]:
# Load the database with the cut-in scenarios.
filename = os.path.join("data", "5_scenarios", "cut_in_scenarios.json")
cutins = DataBaseEmulator(filename)
print("Number of scenarios: {:d}".format(len(cutins.collections["scenario"])))

In [ ]:
# Load the database with the target scenarios.
filename = os.path.join("data", "5_scenarios", "highway_target.json")
targets = DataBaseEmulator(filename)
print("Number of scenarios: {:d}".format(len(targets.collections["scenario"])))

# Obtain parameters

In [ ]:
def cutin_parameters(scenario):
    # Obtain the duration of the lane change.
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    duration = activity.tduration
    ystart = activity.get_state(time=scenario.time["start"])[0]
    yend = activity.get_state(time=scenario.time["end"])[0]
    tstart = activity.tstart
    from_direction = 'r' if activity.name == "left lane change" else 'r'
    
    # Obtain longitudinal speed/position at time of start cut in.
    vstart, vend = None, None
    xstart = None
    init_activity_target = ''
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vstart = activity.get_state(time=scenario.time["start"])[0][0]
            vend = activity.get_state(time=activity.tend)[0][0]
            xstart = activity.get_state(time=scenario.time["start"])[1][0]
            if activity.activity_category.name == "deceleration target":
                init_activity_target = 'd'
            elif activity.activity_category.name == "acceleration target":
                init_activity_target = 'a'
            else:
                init_activity_target = 'c'
            tstartlon = activity.tstart - scenario.time["start"]
            tendlin = activity.tend
            break
            
    # Obtain ego vehicle speed at time of start cut in.
    vego = None
    for actor, activity, _ in scenario.acts:
        if actor.name == "ego vehicle" and \
            activity.activity_category.state == StateVariable.SPEED and \
                activity.tstart <= scenario.time["start"] <= activity.tend:
            vego = activity.get_state(time=scenario.time["start"])[0]
            
    return [duration, ystart, yend, xstart, vstart, vego,
            from_direction, init_activity_target, tstartlon, tendlin]

In [ ]:
# Get parameters of the cut-in: [duration, ystart, yend, xstart]
nscenarios = len(DBE.collections["scenario"])
#pars = [cutin_parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)]
df = pd.DataFrame([cutin_parameters(DBE.get_item("scenario", i)) for i in range(nscenarios)], 
                  columns=['duration', 'ystart', 'yend', 'xstart', 'vstart', 'vego',
                           'from_direction', 'init_activity_target', 'tstartlon', 'tendlin'])
df["vdiff"] = df["vstart"] - df["vego"]
df["ystartabs"] = np.abs(df["ystart"])
df["yspeed"] = (df["yend"] - df["ystart"]) / df["duration"]
df["yspeedabs"] = np.abs(df["yspeed"])
df["tstartlonmap"] = np.log(-df["tstartlon"]+1)

In [ ]:
cutins.get_item("activity", 0)

In [ ]:
def shape_par(activity):
    pars = activity.get_state(time=[activity.tstart, activity.tend])
    return [pars[0], pars[1], activity.tend-activity.tstart]
    
def shape_pars(database: DataBaseEmulator, activity_name):
    activities = []
    for i in range(len(database.collections["activity"])):
        activity = database.get_item("activity", i)
        if activity.name == activity_name:
            activities.append(activity)
    return [shape_par(activity) for activity in activities]

In [ ]:
cruising_cutins = shape_pars(cutins, "cruising target")
cruising_cutins = np.array([np.concatenate((pars[0], [pars[2]])) for pars in cruising_cutins])
cruising_targets = shape_pars(targets, "cruising target")
cruising_targets = np.array([np.concatenate((pars[0], [pars[2]])) for pars in cruising_targets])
cruising_targets = cruising_targets[cruising_targets[:, 0] != cruising_targets[:, 1]]
deceleration_cutins = shape_pars(cutins, "deceleration target")
deceleration_cutins = np.array([np.concatenate((pars[0], [pars[2]])) for pars in deceleration_cutins])
deceleration_targets = shape_pars(targets, "deceleration target")
deceleration_targets = np.array([np.concatenate((pars[0], [pars[2]])) 
                                 for pars in deceleration_targets])
acceleration_cutins = shape_pars(cutins, "acceleration target")
acceleration_cutins = np.array([np.concatenate((pars[0], [pars[2]])) for pars in acceleration_cutins])
acceleration_targets = shape_pars(targets, "acceleration target")
acceleration_targets = np.array([np.concatenate((pars[0], [pars[2]])) 
                                 for pars in acceleration_targets])

In [ ]:
plt.plot(np.sort(cruising_cutins[:, 1]-cruising_cutins[:, 0]), 
         np.linspace(0, 1, len(cruising_cutins)))
plt.plot(np.sort(cruising_targets[:, 1]-cruising_targets[:, 0]), 
         np.linspace(0, 1, len(cruising_targets)))
plt.xlim([-5, 5])

In [ ]:
for i in range(3):
    print(ks_2samp(cruising_cutins[:, i], cruising_targets[:, i]))
print(ks_2samp(cruising_cutins[:, 1]-cruising_cutins[:, 0], 
               cruising_targets[:, 1]-cruising_targets[:, 0]))

In [ ]:
for i in range(3):
    print(ks_2samp(deceleration_cutins[:, i], deceleration_targets[:, i]))

In [ ]:
for i in range(3):
    print(ks_2samp(acceleration_cutins[:, i], acceleration_targets[:, i]))

# Plot a cut-in scenario

In [ ]:
def plot_cutin(scenario, axes=None):
    if axes is None:
        _, axes = plt.subplots(1, 1)
        axes.plot(0, 0, 'gx')
    df = pd.DataFrame(index=np.arange(scenario.time["start"], scenario.time["end"], 0.01),
                      columns=["x", "y", "act"])
    activity = next(activity for activity in scenario.activities 
                    if activity.activity_category.name == 'lane change target')
    df["y"] = activity.get_state(time=df.index.values)
    
    for activity in scenario.activities:
        if activity.activity_category.state == StateVariable.LON_TARGET:
            if activity.tstart < df.index[0]:
                i = 0
            else:
                i = df.index.get_loc(activity.tstart, method='ffill')
            if activity.tend > df.index[-1]:
                j = len(df)
            else:
                j = df.index.get_loc(activity.tend, method='bfill')
            df.loc[df.index[i]:df.index[j-1], "x"] = \
                activity.get_state(time=np.array(df.index[i:j]))[1]
            if activity.activity_category.name == "acceleration target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "a"
            elif activity.activity_category.name == "deceleration target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "d"
            elif activity.activity_category.name == "cruising target":
                df.loc[df.index[i]:df.index[j-1], "act"] = "c"
            else:
                raise ValueError("Unknown longitudinal activity")
                
            if activity.tstart >= df.index[0]:
                axes.plot(df.at[df.index[i], "x"], df.at[df.index[i], "y"], 'k.', ms=10)
                
    for act, color in zip(["a", "d", "c"], ["g", "r", "b"]):
        mask = df["act"] == act
        if np.sum(mask):
            axes.plot(df.loc[mask, "x"], df.loc[mask, "y"], color=color)
    xmax = axes.get_xlim()[1]
    axes.set_xlim([-10, xmax])
    return axes

In [ ]:
axes = plot_cutin(DBE.get_item("scenario", 0))
for i in range(1, 20):
    plot_cutin(DBE.get_item("scenario", i), axes=axes)
plt.xlabel("Longitudinal position [m]")
plt.ylabel("Lateral position [m]")

# Step 1: Initial tags

In [ ]:
for activity, abbr in zip(["acceleration", "deceleration", "cruising"], ["a", "d", "c"]):
    print("Probability of {:12s}: {:4.0f} ({:d}/{:d})%".
          format(activity, np.sum(df["init_activity_target"] == abbr)/nscenarios*100,
                 np.sum(df["init_activity_target"] == abbr), nscenarios))

Furthermore, the initial lateral activity is always a lane change.

# Step 2: Initial parameters

The initial parameters consists of:

- `ystart`: Initial lateral position w.r.t. center lane of ego
- `xstart`: Initial longitudinal offset w.r.t. ego vehicle
- `tlongitudinal`: Time at which the longitudinal activity starts
- `vego`: Initial speed ego vehicle
- `vstart`: Initial speed target at the start of the scenario

In [ ]:
# Determine if the parameters depend on the initial longitudinal activity.
def get_ks_result(act1, act2, signal):
    values1 = df.loc[df["init_activity_target"] == act1, signal].values
    values2 = df.loc[df["init_activity_target"] == act2, signal].values
    result = ks_2samp(values1, values2)
    return result.pvalue

print("     Variable  KS test")
print("                 a-d    a-c    d-c")
for signal in ['ystart', 'yend', 'tstartlon', 'tstartlonmap', 'ystartabs', 'vego', 'vstart']:
    print("{:>13s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        print("  {:5.3f}".format(get_ks_result(act1, act2, signal)), end="")
    print("")

Based on the KS-test, `ystart` is significantly differently distributed in case of acceleration activities. Similarly, `yend` is significantly differently distributed for acceleration activities when compared to cruising activities. When comparing deceleration and cruising activities, the parameter `tstartlon` seems to be differently distributed.

In [ ]:
def test_independent(data1, data2, verbose=False):
    data = np.concatenate((data1, data2))
    k, k1, k2 = KDE(data=data), KDE(data=data1), KDE(data=data2)
    k.compute_bandwidth(), k1.compute_bandwidth(), k2.compute_bandwidth()
    score1 = k.score_leave_one_out(include_const=True)
    score2 = (k1.score_leave_one_out(include_const=True) + 
              k2.score_leave_one_out(include_const=True))
    if verbose:
        print("1 KDE:  {:7.2f}".format(score1))
        print("2 KDes: {:7.2f}".format(score2))
    return score1, score2

print("     Variable  KDE test (positive means similarly distributed)")
print("                    a-d       a-c       d-c")
for signal in ['ystart', 'yend', 'tstartlonmap', 'vego', 'vstart']:
    print("{:>13s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        data1 = df.loc[df["init_activity_target"] == act1, signal].values
        data1 /= np.std(df[signal])
        data2 = df.loc[df["init_activity_target"] == act2, signal].values
        data2 /= np.std(df[signal])
        score1, score2 = test_independent(data1, data2)
        print("  {:8.3f}".format(score1 - score2), end="")
    print("")

In [ ]:
def leave_one_out_score(data):
    kde = KDE(data=data)
    kde.compute_bandwidth()
    return kde.score_leave_one_out(include_const=True)
def compute_score(data, combinations):
    score = 0
    for combination in combinations:
        score += leave_one_out_score(data[:, combination])
    return score
print("a=ystart, b=xstart, c=tstartlon")
combinations = [([0, 1, 2],), ([0, 1], 2), ([0, 2], 1), (0, [1, 2]), (0, 1, 2)]
str_combinations = ['(a,b,c)', '(a,b),(c)', '(a,c),(b)', '(a),(b,c)', '(a),(b),(c)']
for act in ["a", "d", "c"]:
    print("Initial activity: {:s}".format(act))
    data = df.loc[df["init_activity_target"] == act, 
                  ["ystart", "xstart", "tstartlon"]].values
    data /= np.std(data, axis=0)
    scores = [compute_score(data, combination) for combination in combinations]
    i_best = np.argmax(scores)
    for i, (text, score) in enumerate(zip(str_combinations, scores)):
        print("{:>12s}:  {:.3e}{:>10s}".format(text, score, "Best!" if i==i_best else ""))

In [ ]:
spearmanr(data[:, 0], data[:, 1])

Conclusion: The parameters `ystart`, `xstart` and `tstartlon` will be sampled independently from each other. Furthermore, a different KDE is used for each initial longitudinal activity.

But what would the result be if we go through all possibilities?

In [ ]:
plt.plot(data[:, 0], data[:, 2], '.')

In [ ]:
data = df[["ystart", "xstart", "tstartlonmap"]].values
for combination, str_combination in zip(combinations, str_combinations):
    print("{:>12s}: ".format(str_combination), end="")
    total = 0
    for c in combination:
        # Take all
        subdata = data[:, c]
        best = leave_one_out_score(subdata)
        print("{:7.1f} ".format(best))
        
        # Take acceleration apart
        subdata1 = data[:, c][df["init_activity_target"] == "a"]
        subdata2 = data[:, c][np.logical_not(df["init_activity_target"] == "a")]
        score = leave_one_out_score(subdata1) + leave_one_out_score(subdata2)
        print("{:7.1f} ".format(score))
        if score > best:
            best = score

# Parameters activities

For the lane change, we have the parameters `yend` and `duration`. Alternatively, `yspeed` might be used instead of one of these.

For the longitudinal activity, we have the parameters `tend`, `vstart`, and `vend`. 

In [ ]:
print("  Variable  KS test")
print("              a-d    a-c    d-c")
for signal in ['duration', 'yend', 'yspeed']:
    print("{:>10s}".format(signal), end="")
    for act1, act2 in zip(["a", "a", "d"], ["d", "c", "c"]):
        print("  {:5.3f}".format(get_ks_result(act1, act2, signal)), end="")
    print("")

In [ ]:
for act in ['a', 'd', 'c']:
    n = np.sum(df["init_activity_target"] == act)
    plt.plot(np.sort(df.loc[df["init_activity_target"] == act, "ystart"]),
             np.linspace(0, 1, n), label=act)
plt.legend()

In [ ]:
# See if the lane change data is correlated.
print(spearmanr(df["ystart"], df["duration"]))
print(kendalltau(df["ystart"], df["duration"]))
print(spearmanr(df["yend"], df["duration"]))
print(kendalltau(df["yend"], df["duration"]))
print(spearmanr(df["yend"], df["ystart"]))
print(kendalltau(df["yend"], df["ystart"]))

In [ ]:
print(spearmanr(np.abs(df["yend"]), np.abs(df["ystart"])))
print(kendalltau(np.abs(df["yend"]), np.abs(df["ystart"])))

In [ ]:
plt.plot(df["yend"], df["ystart"], '.')

In [ ]:
x = df["ystart"] - df["yend"]
x.loc[df["ystart"] < 0] *= -1
plt.plot(x, '.')

In [ ]:
np.where(np.abs(df["ystart"]) < 1)

In [ ]:
plot_cutin(DBE.get_item("scenario", 46))

In [ ]:
s = DBE.get_item("scenario", 46)

In [ ]:
#plt.plot(s.activities[2].get_state())
d = DataHandler(os.path.join("data", "1_hdf5", "20170530_PP_06_Run_1.hdf5"))

In [ ]:
t = d.targets[s.actors[1].properties["id"]]
plt.plot(t['line_left_next'])
plt.plot(t['line_right_next'])

In [ ]:
plt.plot(d.data["host_lateral_activity"])
plt.xlim([1940, 1960])

In [ ]:
s.actors[1].properties

# Step 4: Next activity

In [ ]:
# Compute 'transition' matrix
A = np.zeros((3, 3))  # 'a', 'd', 'c'
for i in range(nscenarios):
    s = DBE.get_item("scenario", i)
    t_acts = []
    lon_acts = []
    for actor, activity, tstart in s.acts:
        if actor.actor_category.name == "cut-in vehicle":
            if activity.activity_category.state == StateVariable.LON_TARGET:
                t_acts.append(tstart)
                lon_acts.append(activity)
    if len(t_acts) <= 1:
        continue
    i_sorted = np.argsort(t_acts)
    for i, j in zip(i_sorted[:-1], i_sorted[1:]):
        ia = (0 if lon_acts[i].activity_category.name == "acceleration target" else
              1 if lon_acts[i].activity_category.name == "deceleration target" else 2)
        ja = (0 if lon_acts[j].activity_category.name == "acceleration target" else
              1 if lon_acts[j].activity_category.name == "deceleration target" else 2)
        A[ia, ja] += 1

In [ ]:
A

In [ ]:
import ot